# Tweede kamer Topic modeling

In [1]:
import pandas as pd
import numpy as np
import gensim
import plotly.express as px
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

import umap


# Gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel
import pyLDAvis.gensim

## Import data

In [ ]:
## put data chuncks in one zip file
!cat data/CorpusTweedeKamer* > CorpusTweedeKamer.zip

In [3]:
%%time
tweede_kamer = pd.read_csv("CorpusTweedeKamer.zip")
tweede_kamer = (
    tweede_kamer
    .assign(datum = pd.to_datetime(tweede_kamer.date))
    .assign(speaker = tweede_kamer.speaker.str.lower())
)

CPU times: user 13.2 s, sys: 890 ms, total: 14.1 s
Wall time: 14.4 s


In [4]:
tweede_kamer.shape

(1143366, 12)

In [8]:
tweede_kamer.sample(10)

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,datum
274583,2001-12-06,NaN,194,de wit,SP,1363.0,False,27,Maar moet dat niet vaststaan voordat wij hier ...,NL-TweedeKamer,NLD,2001-12-06
958394,2016-10-26,NaN,520,de heer elias,VVD,828.0,False,2,Of houdstermaatschappij.,NL-TweedeKamer,NLD,2016-10-26
356298,2004-04-21,NaN,508,van velzen,SP,1363.0,False,372,Voorzitter. De minister van LNV heeft per brie...,NL-TweedeKamer,NLD,2004-04-21
741743,2012-12-04,NaN,121,anne mulder,VVD,828.0,False,372,"Dat gebeurt al, want sterftecijfers worden ges...",NL-TweedeKamer,NLD,2012-12-04
786064,2013-10-08,NaN,221,klein,other,NaN,False,79,Ik ben in dit verband even de weg kwijt. De he...,NL-TweedeKamer,NLD,2013-10-08
964470,2016-11-16,NaN,276,de heer öztürk,other,NaN,False,5,Dan moet u goed luisteren.,NL-TweedeKamer,NLD,2016-11-16
68623,1996-10-17,NaN,1,rosenmöller,GL,1537.0,False,1352,Voorzitter! De Nederlandse politiek dreigt de ...,NL-TweedeKamer,NLD,1996-10-17
738776,2012-11-21,NaN,291,verheijen,VVD,828.0,False,169,"Het staat verderop in mijn tekst, die ik dan n...",NL-TweedeKamer,NLD,2012-11-21
255890,2001-06-19,NaN,118,voorzitter,other,NaN,True,28,Deze motie is voorgesteld door de leden Passto...,NL-TweedeKamer,NLD,2001-06-19
1084326,2018-10-31,NaN,1056,de heer klaver,GL,1537.0,False,59,Om half één 's nachts zitten er vast hordes me...,NL-TweedeKamer,NLD,2018-10-31


### Embded